# PromptLedger API Demo

This notebook demonstrates how to interact with the PromptLedger API using both operational modes:

1. **Full Mode**: API-based prompt management for prompts managed entirely through the API
2. **Tracking Mode**: Code-based prompts where prompts are defined in application code and the system tracks usage and automatically detects version changes

## Prerequisites

Before running this notebook, ensure you have:

1. **Docker and Docker Compose** installed
2. **Python 3.11+** installed
3. **Required Python packages**: `requests`, `jupyter`

## Setup Instructions

### Step 1: Install Python Dependencies

From the project root directory, run:

```bash
# Install the project with dev dependencies (includes jupyter and requests)
pip install -e ".[dev]"

# Or install just the required packages
pip install requests jupyter
```

### Step 2: Start the PromptLedger API

Make sure you're in the project root directory and run:

```bash
# Start all services (API, PostgreSQL, Redis)
docker-compose up -d

# Check that services are running
docker-compose ps
```

The API will be available at `http://localhost:8000`

You can verify it's running by visiting `http://localhost:8000/docs` in your browser to see the API documentation.

### Step 3: Run this Notebook

```bash
# Launch Jupyter from the project root
jupyter notebook examples/api_demo.ipynb
```

### Stopping the Services

When you're done:

```bash
# Stop all services
docker-compose down

# Stop and remove volumes (cleans database)
docker-compose down -v
```

## Environment Configuration

The API connects to:
- **API**: http://localhost:8000
- **PostgreSQL**: localhost:5432
- **Redis**: localhost:6379

If you need to modify these settings, edit `docker-compose.yml` or create a `.env` file.

---

**Ready to begin!** Execute the cells below in order to see PromptLedger in action.

## Step 1: Import Dependencies

In [21]:
import requests
import json
import subprocess
from typing import Dict, Any, List
from datetime import datetime

# API Base URL
BASE_URL = "http://localhost:8000"
API_V1 = f"{BASE_URL}/v1"

# Helper function to pretty print JSON responses
def print_response(response: requests.Response, title: str = "Response"):
    print(f"\n{'='*60}")
    print(f"{title}")
    print(f"{'='*60}")
    print(f"Status Code: {response.status_code}")
    try:
        print(json.dumps(response.json(), indent=2))
    except:
        print(response.text)
    print(f"{'='*60}\n")

print("✓ Dependencies imported successfully!")

✓ Dependencies imported successfully!


## Step 2: Seed AI Models

Before executing prompts, we need to seed the database with available AI models. This only needs to be done once (or when you reset the database).

In [22]:
# Seed AI models into the database using SQL
print("Seeding AI models into the database...")

# SQL to insert models (uses ON CONFLICT to avoid duplicates)
seed_sql = """
INSERT INTO models (model_id, provider, model_name, max_tokens, supports_streaming, created_at)
VALUES 
    (gen_random_uuid(), 'openai', 'gpt-4o', 128000, true, NOW()),
    (gen_random_uuid(), 'openai', 'gpt-4o-mini', 128000, true, NOW()),
    (gen_random_uuid(), 'openai', 'gpt-4-turbo', 128000, true, NOW()),
    (gen_random_uuid(), 'openai', 'gpt-3.5-turbo', 16384, true, NOW())
ON CONFLICT (provider, model_name) DO NOTHING;
"""

result = subprocess.run(
    ["docker-compose", "exec", "-T", "postgres", "psql", "-U", "postgres", "-d", "prompt_ledger", "-c", seed_sql],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✓ Models seeded successfully!")
else:
    print("Error:", result.stderr)

Seeding AI models into the database...
✓ Models seeded successfully!


## Part 1: Full Mode - API-Based Prompt Management

In **Full Mode**, prompts are managed entirely through the API. This mode is ideal for:
- Business users managing prompts through a UI
- Centralized prompt management
- Prompts that need manual versioning and approval workflows

### 1.1 Create a New Prompt (Full Mode)

In [23]:
# Create a new prompt in full mode
prompt_data = {
    "template_source": "Generate a professional email to {{recipient}} about {{topic}}.",
    "description": "Email generation template",
    "owner_team": "marketing",
    "created_by": "john.doe@example.com",
    "set_active": True  # Set this version as active
}

response = requests.put(
    f"{API_V1}/prompts/email_generator",
    json=prompt_data
)

print_response(response, "Create Prompt (Full Mode)")


Create Prompt (Full Mode)
Status Code: 200
{
  "prompt": {
    "prompt_id": "a438238f-653a-4402-8ae8-919bd0225847",
    "name": "email_generator"
  },
  "version": {
    "version_id": "c70c28db-5538-442d-bb74-70a8d3343c71",
    "version_number": 1
  },
  "version_change": false
}



### 1.2 Get Prompt Details

In [24]:
# Get prompt details including active version
response = requests.get(f"{API_V1}/prompts/email_generator")
print_response(response, "Get Prompt Details")


Get Prompt Details
Status Code: 200
{
  "prompt_id": "a438238f-653a-4402-8ae8-919bd0225847",
  "name": "email_generator",
  "description": "Email generation template",
  "owner_team": "marketing",
  "created_at": "2026-01-20T01:39:38.344411+00:00",
  "updated_at": "2026-01-20T13:45:46.442512+00:00",
  "active_version": {
    "version_id": "c70c28db-5538-442d-bb74-70a8d3343c71",
    "version_number": 1,
    "template_source": "Generate a professional email to {{recipient}} about {{topic}}.",
    "status": "active"
  }
}



### 1.3 Update Prompt (Create New Version)

In [25]:
# Update the prompt template (creates new version)
updated_prompt_data = {
    "template_source": "Generate a professional and friendly email to {{recipient}} about {{topic}}. Keep it concise and actionable.",
    "description": "Enhanced email generation template",
    "owner_team": "marketing",
    "created_by": "jane.smith@example.com",
    "set_active": True
}

response = requests.put(
    f"{API_V1}/prompts/email_generator",
    json=updated_prompt_data
)

print_response(response, "Update Prompt (New Version)")


Update Prompt (New Version)
Status Code: 200
{
  "prompt": {
    "prompt_id": "a438238f-653a-4402-8ae8-919bd0225847",
    "name": "email_generator"
  },
  "version": {
    "version_id": "a3569df8-415c-421b-85a5-d22c90e0bd98",
    "version_number": 2
  },
  "version_change": false
}



### 1.4 List All Versions

In [26]:
# List all versions of the prompt
response = requests.get(f"{API_V1}/prompts/email_generator/versions")
print_response(response, "List Prompt Versions")


List Prompt Versions
Status Code: 200
[
  {
    "version_id": "a3569df8-415c-421b-85a5-d22c90e0bd98",
    "version_number": 2,
    "status": "active",
    "checksum_hash": "de758702d335e16200bc0e7ecc42b26969244ecb50f5f687e9fef5d79c688843",
    "created_by": "jane.smith@example.com",
    "created_at": "2026-01-20T01:40:00.478332+00:00"
  },
  {
    "version_id": "c70c28db-5538-442d-bb74-70a8d3343c71",
    "version_number": 1,
    "status": "active",
    "checksum_hash": "5f109cc82a8278f251162e28aee30fca17401a31e018bf64420894404b91c65a",
    "created_by": "john.doe@example.com",
    "created_at": "2026-01-20T01:39:38.344411+00:00"
  }
]



### 1.5 Execute Prompt (Full Mode)

For full mode prompts, use the generic execution endpoint.

**Note:** This will fail if you don't have an OpenAI API key configured. Set the `OPENAI_API_KEY` environment variable in your `.env` file or docker-compose.yml.

In [27]:
# Execute the prompt synchronously
execution_request = {
    "prompt_name": "email_generator",
    "version_number": None,  # Use active version
    "variables": {
        "recipient": "Sarah",
        "topic": "quarterly sales targets"
    },
    "model": {
        "provider": "openai",
        "model_name": "gpt-4o-mini"
    },
    "params": {
        "temperature": 0.7,
        "max_tokens": 500
    },
    "environment": "production",
    "correlation_id": "demo-exec-001"
}

response = requests.post(
    f"{API_V1}/executions/run",
    json=execution_request
)

print_response(response, "Execute Prompt (Sync)")


Execute Prompt (Sync)
Status Code: 200
{
  "execution_id": "91e684a5-8cf0-4e80-944f-e5908652f0f4",
  "status": "succeeded",
  "mode": "sync",
  "response_text": "Subject: Quarterly Sales Targets Discussion\n\nHi Sarah,\n\nI hope this message finds you well! As we approach the next quarter, I wanted to touch base regarding our sales targets. \n\nCould we schedule a brief meeting to review our goals and strategies? I believe aligning our efforts early will set us up for success. Please let me know your availability this week or next.\n\nLooking forward to your thoughts!\n\nBest,  \n[Your Name]  \n[Your Position]  \n[Your Contact Information]  ",
  "telemetry": {
    "prompt_tokens": 26,
    "response_tokens": 98,
    "latency_ms": 2706
  }
}



### 1.6 Execute Prompt Asynchronously

In [28]:
# Submit prompt for async execution
response = requests.post(
    f"{API_V1}/executions/submit",
    json=execution_request
)

print_response(response, "Submit Async Execution")

# Save execution ID for later
if response.status_code == 200:
    execution_id = response.json().get("execution_id")
    print(f"Execution ID: {execution_id}")


Submit Async Execution
Status Code: 200
{
  "execution_id": "44a79c9f-8e37-474e-acc8-68cdb818f730",
  "status": "queued",
  "mode": "async"
}

Execution ID: 44a79c9f-8e37-474e-acc8-68cdb818f730


### 1.7 Check Execution Status

In [29]:
# Check execution status (use execution_id from previous step)
if 'execution_id' in locals():
    response = requests.get(f"{API_V1}/executions/{execution_id}")
    print_response(response, "Execution Status")
else:
    print("No execution_id available. Run the async execution cell first.")


Execution Status
Status Code: 200
{
  "execution_id": "44a79c9f-8e37-474e-acc8-68cdb818f730",
  "status": "succeeded",
  "mode": "async",
  "environment": "production",
  "created_at": "2026-01-20T13:46:13.375016+00:00",
  "response_text": "Subject: Quarterly Sales Targets Discussion\n\nHi Sarah,\n\nI hope this message finds you well! As we approach the end of the quarter, I wanted to touch base regarding our sales targets. \n\nCould we schedule a brief meeting to review our progress and discuss strategies for achieving our goals? I believe a focused discussion will help us align our efforts effectively.\n\nPlease let me know your availability this week or next, and I\u2019ll do my best to accommodate.\n\nLooking forward to our conversation!\n\nBest,  \n[Your Name]  \n[Your Position]  \n[Your Contact Information]  ",
  "completed_at": "2026-01-20T13:46:16.011932+00:00",
  "telemetry": {
    "prompt_tokens": 26,
    "response_tokens": 114,
    "latency_ms": 2595
  }
}



## Part 2: Tracking Mode - Code-Based Prompt Management

In **Tracking Mode**, prompts are defined in application code and the system:
- Automatically tracks version changes based on template content
- Detects when templates change and creates new versions
- Ideal for engineering teams managing prompts in code
- Supports GitOps workflows

### 2.1 Register Code-Based Prompts

This simulates what happens when your application starts up and registers its prompts.

In [30]:
# Register multiple code-based prompts at once
registration_request = {
    "prompts": [
        {
            "name": "WELCOME_MESSAGE",
            "template_source": "Hello {{name}}! Welcome to {{service}}."
        },
        {
            "name": "ERROR_EXPLANATION",
            "template_source": "Explain this error to a user: {{error_message}}"
        },
        {
            "name": "CODE_REVIEW",
            "template_source": "Review the following {{language}} code and provide constructive feedback:\n\n{{code}}"
        }
    ]
}

response = requests.post(
    f"{API_V1}/prompts/register-code",
    json=registration_request
)

print_response(response, "Register Code Prompts")


Register Code Prompts
Status Code: 200
{
  "registered": [
    {
      "name": "WELCOME_MESSAGE",
      "mode": "tracking",
      "version": 1,
      "change_detected": false,
      "previous_version": null
    },
    {
      "name": "ERROR_EXPLANATION",
      "mode": "tracking",
      "version": 1,
      "change_detected": false,
      "previous_version": null
    },
    {
      "name": "CODE_REVIEW",
      "mode": "tracking",
      "version": 1,
      "change_detected": false,
      "previous_version": null
    }
  ]
}



### 2.2 Register Again (No Changes)

When you register the same prompts again with no changes, the system detects this and doesn't create new versions.

In [31]:
# Register again with same content - should detect no changes
response = requests.post(
    f"{API_V1}/prompts/register-code",
    json=registration_request
)

print_response(response, "Register Again (No Changes)")


Register Again (No Changes)
Status Code: 200
{
  "registered": [
    {
      "name": "WELCOME_MESSAGE",
      "mode": "tracking",
      "version": 1,
      "change_detected": false,
      "previous_version": null
    },
    {
      "name": "ERROR_EXPLANATION",
      "mode": "tracking",
      "version": 1,
      "change_detected": false,
      "previous_version": null
    },
    {
      "name": "CODE_REVIEW",
      "mode": "tracking",
      "version": 1,
      "change_detected": false,
      "previous_version": null
    }
  ]
}



### 2.3 Register with Changes

When you modify a template and register again, the system automatically creates a new version.

In [32]:
# Update one of the prompts
updated_registration = {
    "prompts": [
        {
            "name": "WELCOME_MESSAGE",
            "template_source": "Hello {{name}}! Welcome to {{service}}. We're glad you're here!"
        }
    ]
}

response = requests.post(
    f"{API_V1}/prompts/register-code",
    json=updated_registration
)

print_response(response, "Register with Changes")


Register with Changes
Status Code: 200
{
  "registered": [
    {
      "name": "WELCOME_MESSAGE",
      "mode": "tracking",
      "version": 2,
      "change_detected": false,
      "previous_version": null
    }
  ]
}



### 2.4 Execute Code-Based Prompt (Sync)

In [33]:
# Execute a tracking mode prompt synchronously
tracking_execution_request = {
    "variables": {
        "name": "Alice",
        "service": "PromptLedger"
    },
    "model_name": "gpt-4o-mini",
    "mode": "sync",
    "params": {
        "temperature": 0.5,
        "max_tokens": 100
    }
}

response = requests.post(
    f"{API_V1}/prompts/WELCOME_MESSAGE/execute",
    json=tracking_execution_request
)

print_response(response, "Execute Code Prompt (Sync)")


Execute Code Prompt (Sync)
Status Code: 200
{
  "execution_id": "504df1c1-5730-43f0-9d54-df22989977e0",
  "status": "succeeded",
  "mode": "sync",
  "response_text": "Hello! Thank you for the warm welcome! I'm excited to be here. How can I assist you today?",
  "telemetry": {
    "prompt_tokens": 20,
    "response_tokens": 22,
    "latency_ms": 1259
  },
  "prompt_mode": "tracking"
}



### 2.5 Execute Code-Based Prompt (Async)

In [34]:
# Execute a tracking mode prompt asynchronously
tracking_async_request = {
    "variables": {
        "language": "Python",
        "code": "def add(a, b):\n    return a + b\n\nresult = add(2, 2)"
    },
    "model_name": "gpt-4o-mini",
    "mode": "async",
    "params": {
        "temperature": 0.7,
        "max_tokens": 500
    }
}

response = requests.post(
    f"{API_V1}/prompts/CODE_REVIEW/execute",
    json=tracking_async_request
)

print_response(response, "Execute Code Prompt (Async)")

# Save execution ID
if response.status_code == 200:
    code_execution_id = response.json().get("execution_id")
    print(f"Code Execution ID: {code_execution_id}")


Execute Code Prompt (Async)
Status Code: 200
{
  "execution_id": "18508c78-3aa4-45f1-9885-730671346ffb",
  "status": "queued",
  "mode": "async",
  "prompt_mode": "tracking"
}

Code Execution ID: 18508c78-3aa4-45f1-9885-730671346ffb


#### Check Status

In [39]:
# Check execution status (use execution_id from previous step)
if 'execution_id' in locals():
    response = requests.get(f"{API_V1}/executions/{code_execution_id}")
    print_response(response, "Execution Status")
else:
    print("No execution_id available. Run the async execution cell first.")


Execution Status
Status Code: 200
{
  "execution_id": "18508c78-3aa4-45f1-9885-730671346ffb",
  "status": "succeeded",
  "mode": "async",
  "environment": "dev",
  "created_at": "2026-01-20T13:46:53.867896+00:00",
  "response_text": "The provided Python code is simple and effectively demonstrates the functionality of adding two numbers. Here are some constructive feedback points to consider for improvement and best practices:\n\n1. **Function Documentation**: It's a good practice to include a docstring for your function to explain its purpose, parameters, and return value. This makes the code easier to understand for others (and yourself in the future).\n\n   ```python\n   def add(a, b):\n       \"\"\"\n       Adds two numbers together.\n\n       Parameters:\n       a (int or float): The first number.\n       b (int or float): The second number.\n\n       Returns:\n       int or float: The sum of a and b.\n       \"\"\"\n       return a + b\n   ```\n\n2. **Type Annotations**: Adding t

### 2.6 View Prompt History (Tracking Mode)

In [40]:
# View version history for a code-based prompt
response = requests.get(f"{API_V1}/prompts/WELCOME_MESSAGE/versions")
print_response(response, "Prompt Versions (Tracking Mode)")


Prompt Versions (Tracking Mode)
Status Code: 200
[
  {
    "version_id": "19962dbe-5708-436f-bae1-e89c0102c6b8",
    "version_number": 2,
    "status": "active",
    "checksum_hash": "0a979face33667be5d41a503d3f799702f030f0dded5986ff0a0e0fa237285b4",
    "created_by": null,
    "created_at": "2026-01-20T02:00:59.527073+00:00"
  },
  {
    "version_id": "e010904b-7f09-40e3-88dd-47075d67906f",
    "version_number": 1,
    "status": "active",
    "checksum_hash": "439b0d48ebaa5631eb5a02fbc9284a3f883f7d6601db54a44d39e9c9f360ff08",
    "created_by": null,
    "created_at": "2026-01-20T02:00:26.671100+00:00"
  }
]



## Part 3: Analytics and Monitoring

PromptLedger provides unified analytics across both modes.

### 3.1 Get Overall Analytics

In [41]:
# Get analytics across all modes
response = requests.get(f"{API_V1}/analytics/prompts?mode=all")
print_response(response, "Overall Analytics")


Overall Analytics
Status Code: 200
{
  "summary": {
    "total_executions": 23,
    "full_mode_prompts": 2,
    "tracking_mode_prompts": 6
  },
  "by_mode": {
    "full": {
      "execution_count": 15,
      "avg_latency_ms": 4445
    },
    "tracking": {
      "execution_count": 8,
      "avg_latency_ms": 6510
    }
  }
}



### 3.2 Get Analytics by Mode

In [42]:
# Get analytics for full mode prompts only
response = requests.get(f"{API_V1}/analytics/prompts?mode=full")
print_response(response, "Full Mode Analytics")

# Get analytics for tracking mode prompts only
response = requests.get(f"{API_V1}/analytics/prompts?mode=tracking")
print_response(response, "Tracking Mode Analytics")


Full Mode Analytics
Status Code: 200
{
  "mode": "full",
  "prompt_count": 2,
  "execution_count": 15,
  "avg_latency_ms": 4445,
  "total_prompt_tokens": 265,
  "total_response_tokens": 1445
}


Tracking Mode Analytics
Status Code: 200
{
  "mode": "tracking",
  "prompt_count": 6,
  "execution_count": 8,
  "avg_latency_ms": 6510,
  "total_prompt_tokens": 156,
  "total_response_tokens": 1407
}



### 3.3 List Recent Executions

In [44]:
# List recent executions
response = requests.get(f"{API_V1}/executions/?limit=10")
print_response(response, "Recent Executions")


Recent Executions
Status Code: 500
Internal Server Error



### 3.4 Filter Executions by Prompt

In [ ]:
# List executions for a specific prompt
response = requests.get(f"{API_V1}/executions/?prompt_name=WELCOME_MESSAGE&limit=5")
print_response(response, "Executions for WELCOME_MESSAGE")

## Part 4: Real-World Workflow Examples

### 4.1 Full Mode Workflow: Business User Updates Marketing Prompt

In [ ]:
# Step 1: Create initial version
prompt_v1 = {
    "template_source": "Create a marketing message for {{product}} targeting {{audience}}.",
    "description": "Marketing message generator",
    "owner_team": "marketing",
    "created_by": "marketing.team@example.com",
    "set_active": True
}

response = requests.put(f"{API_V1}/prompts/marketing_message", json=prompt_v1)
print_response(response, "Step 1: Create Initial Version")

# Step 2: Business user tests and refines
prompt_v2 = {
    "template_source": "Create an engaging marketing message for {{product}} that resonates with {{audience}}. Focus on benefits and use emotional language.",
    "description": "Enhanced marketing message generator",
    "owner_team": "marketing",
    "created_by": "marketing.team@example.com",
    "set_active": False  # Draft version
}

response = requests.put(f"{API_V1}/prompts/marketing_message", json=prompt_v2)
print_response(response, "Step 2: Create Draft Version")

# Step 3: Test the new version
test_execution = {
    "prompt_name": "marketing_message",
    "version_number": 2,  # Specify version 2 for testing
    "variables": {
        "product": "CloudSync Pro",
        "audience": "small business owners"
    },
    "model": {"provider": "openai", "model_name": "gpt-4o-mini"},
    "environment": "staging"
}

response = requests.post(f"{API_V1}/executions/run", json=test_execution)
print_response(response, "Step 3: Test Draft Version")

# Step 4: Promote to active
prompt_v2["set_active"] = True
response = requests.put(f"{API_V1}/prompts/marketing_message", json=prompt_v2)
print_response(response, "Step 4: Promote to Active")

### 4.2 Tracking Mode Workflow: Developer Updates Code Prompts

In [ ]:
# Simulating application startup - register prompts from code
class PromptRegistry:
    """Example: How prompts might be defined in application code"""
    
    PROMPTS = {
        "USER_ONBOARDING": "Welcome {{user_name}}! Let's get you started with {{feature}}.",
        "FEATURE_ANNOUNCEMENT": "Exciting news! We've just released {{feature_name}}. {{description}}",
        "SUPPORT_TICKET": "Help the user resolve this issue: {{issue_description}}"
    }
    
    @classmethod
    def register_all(cls):
        """Register all prompts with PromptLedger"""
        prompts_list = [
            {"name": name, "template_source": template}
            for name, template in cls.PROMPTS.items()
        ]
        return {"prompts": prompts_list}

# Initial registration (e.g., app startup)
registration = PromptRegistry.register_all()
response = requests.post(f"{API_V1}/prompts/register-code", json=registration)
print_response(response, "Initial App Registration")

# Developer updates prompt in code and redeploys
PromptRegistry.PROMPTS["USER_ONBOARDING"] = "Welcome {{user_name}}! We're thrilled to have you. Let's explore {{feature}} together."

registration = PromptRegistry.register_all()
response = requests.post(f"{API_V1}/prompts/register-code", json=registration)
print_response(response, "After Code Update (New Version Detected)")

# View history to see automatic versioning
response = requests.get(f"{API_V1}/prompts/USER_ONBOARDING/versions")
print_response(response, "Automatic Version History")

## Summary

### Full Mode
- **Use when**: Business users need to manage prompts through UI/API
- **Key endpoints**:
  - `PUT /v1/prompts/{name}` - Create/update prompts
  - `GET /v1/prompts/{name}` - Get prompt details
  - `GET /v1/prompts/{name}/versions` - List all versions
  - `POST /v1/executions/run` - Execute synchronously
  - `POST /v1/executions/submit` - Execute asynchronously

### Tracking Mode
- **Use when**: Engineering teams manage prompts in code
- **Key endpoints**:
  - `POST /v1/prompts/register-code` - Register code prompts
  - `POST /v1/prompts/{name}/execute` - Execute prompt
  - `GET /v1/prompts/{name}/versions` - View version history

### Both Modes Support
- Automatic version detection
- Execution telemetry and analytics
- Sync and async execution
- Full audit trail
- Unified `/versions` endpoint for viewing version history